<a href="https://colab.research.google.com/github/Ritwik411/Movie_Recommendation_System/blob/master/AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.chdir("/content/drive/My Drive/Boltzman_Machine")

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [91]:
df_train = pd.read_csv('ml-100k/u1.base',delimiter='\t')
df_test = pd.read_csv('ml-100k/u1.test',delimiter='\t')

In [92]:
df_train.columns = ['user_id','movie_id','rating','timestamp']
df_test.columns = df_train.columns

In [74]:
# train.head()

In [75]:
# movie_names = pd.read_csv("ml-100k/movies.csv")
# movie_names.head()

In [76]:
# l1 = movie_names['genres'].str.split("|")
# s = set()
# for i in l1:
#   for j in i:
#     if j not in s:
#       s.add(j)
# print(s)

In [77]:
# list(s)

In [78]:
# l1

In [79]:
# temp = movie_names['genres'].str.split("|",expand=True)

In [80]:
# temp.head()

In [81]:
train = np.array(df_train,dtype='int')
test = np.array(df_test,dtype='int')

In [82]:
num_users = int(max(max(train[:,0]),max(test[:,0])))
num_movies = int(max(max(train[:,1]),max(test[:,1])))

In [83]:
print(num_users,num_movies)

943 1682


In [48]:
def convert(data):
  new_data = []
  for user_id in range(1,num_users+1):
    movie_id = data[:,1][data[:,0]==user_id]
    rating_id = data[:,2][data[:,0]==user_id]
    ratings = np.zeros(num_movies)
    ratings[movie_id-1] = rating_id
    new_data.append(list(ratings))
  return new_data

In [84]:
train = convert(train) #Size of train - 943 rows(number of users), 1682 columns (rating given to each movie)
test = convert(test)

In [50]:
train = torch.FloatTensor(train)
test = torch.FloatTensor(test)

In [51]:
class SAE(nn.Module):
  def __init__(self,):
    super(SAE,self).__init__()
    #No. of hidden layers in the neural network
    self.fc1 = nn.Linear(num_movies,20)
    self.fc2 = nn.Linear(20,10)
    self.fc3 = nn.Linear(10, 20)
    self.fc4 = nn.Linear(20,num_movies)
    #Activation function usign Sigmoid
    self.activation = nn.Sigmoid()
  #Encoding and decoding
  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(),lr = 0.01, weight_decay = 0.5)

In [ ]:
#Training
nb_epoch = 256
for epoch in range(1,nb_epoch+1):
  train_loss = 0
  s = 0.
  for user_id in range(num_users):
    input = Variable(train[user_id]).unsqueeze(0) # creates a batch
    target = input.clone()
    if torch.sum(target.data>0)>0:
      output = sae(input)
      target.require_grad = False
      output[target ==0] = 0
      loss = criterion(output, target)
      mean_corrector = num_movies/float(torch.sum(target.data>0)+ 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print("epoch : "+ str(epoch)+" loss :"+str(train_loss/s))

In [53]:
test_loss = 0
s = 0.
for user_id in range(num_users):
  input = Variable(train[user_id]).unsqueeze(0) #Keep train set
  target = Variable(test[user_id])
  if torch.sum(target.data>0)>0:
    output = sae(input)
    target.require_grad = False
    output[(target ==0).unsqueeze(0)] = 0 #Non-zero ratings
    loss = criterion(output, target)
    mean_corrector = num_movies/float(torch.sum(target.data>0)+ 1e-10)
    #No back propogation using testing
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
    # No optimizer.step() for testing
print("Test Loss : "+str(test_loss/s))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:445: UserWarning: Using a target size (torch.Size([1682])) that is different to the input size (torch.Size([1, 1682])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test Loss : tensor(0.9488)


In [64]:
# nar = torch.detach(sae(input)).numpy()

In [70]:
# temp = []
# for i in nar:
#   for j in i:
#     temp.append(j)